# The `super` builtin function

The builtin function `super` is used when we want to call a method from a base class from the derived class. For example:

```python
class LastUpdatedOrderedDict(OrderedDict):
    """Stores items keeping the update order"""
    

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        self.move_to_end(key)

# We could do the above like this
class NotRecommended(OrderedDict):
    """A counter example"""

    def __setitem__(self, key, value):
        OrderedDict.__setitem__(self, key, value)
        self.move_to_end(key)
```

The counterexample above works for this particular case, but it's not recommended because we need to repeat the base class name. Calling `super()` is used because it supports multiple inheritance.

The most common use of the `super()` function is in constructors, where we use it to call the `__init__()` function from the base class in the derived class.

```python
class MyClass(BaseClass):
    def __init__(self, a, b):
        super().__init__(a, b) # instantiate base class attributes
```

The `super()` has two arguments that are inferred by the interpreter in most cases, but we might need to use them in special cases, such as, bypassing the method resolution order (MRO), for testes, and so on. The arguments are:

- `type`: The name of the superclass, or the beginning of the path to the desired method
- `object_or_type`: The object that will be called (for instance methods)  or the class (for class methods). By default, it is `self` when the call is from within a instance method.

# Subclasses of builtin types (danger)

Since python 2.2, we can have a subclass of a builtin type, like a `dict`, a `str` or a `list`. However, the CPython will not normally call the overloaded methods of the builtin types, as seen below. This could lead to various unknown behaviors. This is because the builtin types calls direct the C interface below instead of a class defined in python. The solution to this problem is to inherit from `UserDict`, `UserStr` and `UserList`, available at `collections` lib, instead of the builtin type.

In [3]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict(one=1)
dd # {'one': 1}. The constructor ignores __setitem__ modification

dd['two'] = 2
dd # {'one': 1, 'two': [2, 2]}. here the modified __setitem__ is called

dd.update(three=3) # Also ignores modified __setitem__
dd # {'one': 1, 'two': [2, 2], 'three': 3}


class AnswerDict(dict):
    def __getitem__(self, key):
        return 42
    
ad = AnswerDict(a='foo')
ad['a'] # 42

d = {}
d.update(ad)

d['a'] # 'foo' # Modified __getitem__ is ignored.
d # {'a': 'foo'}

{'one': 1, 'two': [2, 2], 'three': 3}

# Multiple inheritance

Python, like C++ and unlike Java, supports multiple inheritance, which means a class can be derived from two or more base classes. Multiple inheritance comes with a problem: naming conflicts that might occur when superclasses have methods with the same name. This is called the diamond problem. For example:

In [8]:
class Root:
    def ping(self):
        print(f'{self}.ping() in Root')
    def pong(self):
        print(f'{self}.pong() in Root')
    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'

class A(Root):
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()
    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()

class B(Root):
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()
    def pong(self):
        print(f'{self}.pong() in B')

class Leaf(A, B):
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

leaf1 = Leaf()
leaf1.ping()
# <instance of Leaf>.ping() in Leaf
# <instance of Leaf>.ping() in A
# <instance of Leaf>.ping() in B
# <instance of Leaf>.ping() in Root

leaf1.pong()
# <instance of Leaf>.pong() in A
# <instance of Leaf>.pong() in B



Leaf.__mro__

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root
<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

The Root.pong() is not called because B.pong() does not call super().

Question: the book say that A.pong() calls B.pong() but that's
not what it seems to be. It seems that Leaf.pong() calls B.pong().

The book answer the question afterwards.

The `Leaf.__mro__` shows the order that `super()` will obey when searching for the methods. It uses the C3 algorithm to determine it's order.

The order of the classes in the class declaration matters. If we have done `class(B, A)` instead of `class(A, B)` then, the `B.pong()` would be called first in the second example. And because this method does not call `super()` then the chain of inheritance ends there.

A method that calls `super()` is called a cooperative method. A non cooperative method in a derived class might be the source of bugs, because any user would expect that `leaf1.pong()` would call `Root.pong()` and that did not happen.

Cooperative methods must have compatible signature, because we never know if `A.ping()` would be called before or after `B.ping()`. The sequence of the calls depends on the MRO. Another example:

```python
class U():
    def ping(self):
        print(f'{self}.ping() in U')
        super().ping()

class LeafUA(U, A):
    def ping(self):
        print(f'{self}.ping() in U')

LeafUA.__mro__
(LeafUA, U, A, Root, object)

u = U()
u.ping() # Unknown ping in super()

leaf2 = LeafUA()
leaf2.ping()
# <instance of LeafUA>.ping() in LeafUA
# <instance of LeafUA>.ping() in U
# <instance of LeafUA>.ping() in A
# <instance of LeafUA>.ping() in Root
```

If `class LeafAU(A, U)` would be used, then the call to `A.ping()` would call directly `Root.ping()`